In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox

# تحميل البيانات
distance_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/distance.csv")
orders_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/order_small.csv")
orders_df = pd.read_csv("C:/Users/Acer/OneDrive/Desktop/order_large.csv")
# إعداد خرائط المدن
cities = pd.unique(distance_df[['Source', 'Destination']].values.ravel())
city_to_index = {city: idx for idx, city in enumerate(cities)}
index_to_city = {idx: city for city, idx in city_to_index.items()}

# بناء مصفوفة المسافات
n_cities = len(cities)
distance_matrix = np.full((n_cities, n_cities), np.inf)
for _, row in distance_df.iterrows():
    i = city_to_index[row['Source']]
    j = city_to_index[row['Destination']]
    d = row['Distance(M)']
    distance_matrix[i][j] = d
    distance_matrix[j][i] = d

# خوارزمية النمل ACO
class AntColony:
    def __init__(self, distances, n_ants, n_best, n_iterations, decay, alpha=1, beta=2):
        self.distances = distances
        self.pheromone = np.ones(self.distances.shape) / len(distances)
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.all_inds = range(len(distances))

    def run(self):
        best = ([], np.inf)
        for _ in range(self.n_iterations):
            all_paths = self.gen_all_paths()
            self.spread_pheromones(all_paths)
            shortest_path = min(all_paths, key=lambda x: x[1])
            if shortest_path[1] < best[1]:
                best = shortest_path
            self.pheromone *= (1 - self.decay)
        return best

    def spread_pheromones(self, paths):
        paths = sorted(paths, key=lambda x: x[1])
        for path, dist in paths[:self.n_best]:
            for move in path:
                self.pheromone[move] += 1.0 / dist

    def gen_path(self, start):
        path = []
        visited = set([start])
        prev = start
        for _ in range(len(self.distances) - 1):
            move = self.pick_move(prev, visited)
            path.append((prev, move))
            visited.add(move)
            prev = move
        path.append((prev, start))
        return path

    def pick_move(self, current, visited):
        pheromone = np.copy(self.pheromone[current])
        pheromone[list(visited)] = 0
        heuristic = 1 / (self.distances[current] + 1e-10)
        heuristic[list(visited)] = 0
        prob = (pheromone ** self.alpha) * (heuristic ** self.beta)
        prob /= prob.sum()
        return np.random.choice(self.all_inds, 1, p=prob)[0]

    def gen_all_paths(self):
        paths = []
        for _ in range(self.n_ants):
            path = self.gen_path(0)
            dist = sum(self.distances[i][j] for i, j in path)
            paths.append((path, dist))
        return paths

# دالة تنفيذ التحسين عند الضغط على الزر
def run_optimization():
    try:
        max_weight_kg = float(weight_entry.get())
        max_weight = max_weight_kg * 1000

        truck_id = 1
        current_weight = 0
        truck_assignments = {}
        current_group = []

        for _, row in orders_df.iterrows():
            weight = row['Weight']
            if current_weight + weight > max_weight:
                truck_assignments[f"Truck_{truck_id}"] = current_group
                truck_id += 1
                current_weight = 0
                current_group = []
            current_group.append(row)
            current_weight += weight

        if current_group:
            truck_assignments[f"Truck_{truck_id}"] = current_group

        output_text.delete('1.0', tk.END)
        output_text.insert(tk.END, f"\n🚚 Number of Trucks Used: {len(truck_assignments)}\n", 'title')
        output_text.insert(tk.END, "===============================\n", 'line')

        total_distance = 0

        
        
        for truck, orders in truck_assignments.items():
            city_indices = [city_to_index[row['Source']] for row in orders if row['Source'] in city_to_index]
            if not city_indices:
                continue

            # Prepare submatrix for ACO
            to_keep = [0] + city_indices  # Assuming 0 is the warehouse or start point
            submatrix = distance_matrix[np.ix_(to_keep, to_keep)]
            local_index_to_city = {i: index_to_city[j] for i, j in enumerate(to_keep)}

            aco = AntColony(
                distances=submatrix,
                n_ants=10, n_best=3, n_iterations=30, decay=0.1, alpha=1, beta=2
            )
            best_path, best_cost = aco.run()
            city_names = [local_index_to_city[i] for i, _ in best_path] + [local_index_to_city[best_path[-1][1]]]

            total_weight = sum([r['Weight'] for r in orders])
            order_ids = [str(r['Order_ID']) for r in orders]

            output_text.insert(tk.END, f"\n🛻 {truck}\n", 'subtitle')
            output_text.insert(tk.END, f"   Orders: {', '.join(order_ids)}\n", 'text')
            output_text.insert(tk.END, f"   Total Weight: {total_weight/1000:.2f} kg\n", 'text')
            output_text.insert(tk.END, f"   Optimized Route: {' -> '.join(city_names)}\n", 'route')
            output_text.insert(tk.END, f"   Min Cost (Distance): {best_cost:.2f} meters\n", 'text')

            total_distance += best_cost

        output_text.insert(tk.END, f"\n📏 Total Minimum Delivery Cost (Distance) for All Trucks: {total_distance:.2f} meters\n", 'title')

    except ValueError:
        messagebox.showerror("Input Error", "Please enter a valid number for weight")

# واجهة المستخدم (Tkinter)
root = tk.Tk()
root.title("Vehicle Routing Problem Optimizer")
root.geometry("900x700")
root.configure(bg="#f0f4f8")

frame = tk.Frame(root, bg="#ffffff", padx=20, pady=20, relief=tk.RAISED, bd=2)
frame.pack(pady=20)

header = tk.Label(frame, text="🚚 VRP using Ant Colony Optimization", font=("Helvetica", 16, "bold"), bg="#ffffff", fg="#333399")
header.pack(pady=10)

weight_label = tk.Label(frame, text="Enter Max Truck Weight (kg):", font=("Helvetica", 12), bg="#ffffff")
weight_label.pack(pady=5)
weight_entry = tk.Entry(frame, font=("Helvetica", 12), width=12)
weight_entry.pack(pady=5)

run_button = tk.Button(frame, text="Optimize Routes", command=run_optimization, bg="#4CAF50", fg="white", font=("Helvetica", 12, "bold"))
run_button.pack(pady=15)

output_text = tk.Text(root, height=28, width=100, font=("Consolas", 10), bg="#e6f2ff", fg="#000000", wrap=tk.WORD)
output_text.pack(pady=10)

# أنماط النصوص للألوان والترتيب
output_text.tag_configure('title', font=('Helvetica', 12, 'bold'), foreground='#1a237e')
output_text.tag_configure('subtitle', font=('Helvetica', 11, 'bold'), foreground='#0d47a1')
output_text.tag_configure('text', font=('Consolas', 10), foreground='#263238')
output_text.tag_configure('route', font=('Courier', 10, 'bold'), foreground='#388e3c')
output_text.tag_configure('line', foreground='#9e9e9e')

root.mainloop()